In [6]:
import os
import shutil
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from keras.models import Model
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import MaxPooling2D
from keras.layers import Input
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.models import load_model
from keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt

MODELS_DIR = os.path.abspath("C:/Projects/tensorflow/models")
MODEL_FILE_PATH = os.path.join(MODELS_DIR, "cifar10_model.h5")
FULL_MODEL_FILE_PATH = os.path.join(MODELS_DIR, "full_cifar10_model")
LOGS_DIR = os.path.abspath("C:/Projects/tensorflow/logs")
MODEL_LOGS_DIR = os.path.join(LOGS_DIR, "cifar10_logs")

Preprocessing

In [7]:
def get_dataset(num_classes: int) -> tuple[tuple[np.ndarray, np.ndarray], tuple[np.ndarray, np.ndarray], tuple[np.ndarray, np.ndarray]]:
    (x_train, y_train), (x_val_, y_val_) = cifar10.load_data()
    x_train = x_train.astype(np.float32)
    x_val_ = x_val_.astype(np.float32)
    x_val, x_test, y_val, y_test = train_test_split(x_val_, y_val_, test_size=0.66)
    x_test = x_test.astype(np.float32)
    
    # normalize
    x_train = x_train/255.0
    x_val = x_val/255.0
    x_test = x_test/255.0

    # Data Augmentation
    augment_size = 5_000
    image_gen = ImageDataGenerator(
        rotation_range=5,
        zoom_range=0.05,
        width_shift_range=0.08,
        height_shift_range=0.08
    )
    image_gen.fit(x_train, augment=True)
    rndm_idxs = np.random.randint(low=x_train.shape[0],high=None, size=augment_size)
    x_augmented = x_train[rndm_idxs].copy()
    y_augmented = y_train[rndm_idxs].copy()
    x_augmented = image_gen.flow(x_augmented, np.zeros(augment_size), batch_size=augment_size, shuffle=False).next()[0]
    x_train = np.concatenate((x_train, x_augmented))
    y_train = np.concatenate((y_train, y_augmented))
    
    y_train = to_categorical(y_train, num_classes=num_classes, dtype=np.float32)
    y_val = to_categorical(y_val, num_classes=num_classes, dtype=np.float32)
    y_test = to_categorical(y_test, num_classes=num_classes, dtype=np.float32)
    print (f"x shape: {x_train.shape}   y shape: {y_train.shape}")
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)


Get DATA

In [8]:
num_classes = 10
img_shape = (32, 32, 3)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = get_dataset(num_classes=num_classes)

x shape: (55000, 32, 32, 3)   y shape: (55000, 10)


In [9]:
def build_model(img_shape : tuple[int,int,int], num_classes: int, filters1: int, filters2: int,
                filters3: int,kernel1: int, kernel2: int, kernel3: int, dense_layer: int,
                learning_rate: float, optimizer: tf.keras.optimizers.Optimizer) -> Model:
    
    img_input = Input(img_shape)
    x = Conv2D(filters=filters1, kernel_size=kernel1, padding = "same")(img_input)
    x = Activation("relu")(x)
    x = Conv2D(filters=filters2, kernel_size=kernel2)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D()(x)
    x = Conv2D(filters=filters3, kernel_size=kernel3)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D()(x)
     
    x = Flatten()(x)
    x = Dense(units=dense_layer)(x)
    x = Activation("relu")(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation("softmax")(x)
    
    model = Model(
        inputs=[img_input],
        outputs=[y_pred]
    )
    
    model.compile(loss="categorical_crossentropy",
              optimizer=optimizer(learning_rate=learning_rate),
              metrics=["accuracy"])
    
    return model

Build Model with GridSearch

Train the Model

In [ ]:
param_grid = {
    "filters1": [32], "filters2": [32,64], "filters3": [64, 128],
    "kernel1": [3,5], "kernel2": [3,5], "kernel3": [7],
    "dense_layer": [512], "learning_rate": [0.001, 0.0005],
    "optimizer": [Adam, RMSprop]
}

results = {
    "best_score": -np.inf,
    "best_params": {},
    "val_scores": [],
    "params": []
}

grid = ParameterGrid(param_grid= param_grid)

for idx, param in enumerate(grid):
    print(f"{idx}.Combination Run:")
    model = build_model(img_shape=img_shape,num_classes=num_classes, **param)

    model_log_dir = os.path.join(LOGS_DIR, f"modelGrid{idx}")
    if os.path.exists(model_log_dir):
        shutil.rmtree(model_log_dir)
        os.mkdir(model_log_dir)
        
    tb_callback = TensorBoard(
        log_dir=model_log_dir,
        histogram_freq=0
    )

    history = model.fit (x=x_train, 
            y=y_train, 
            epochs=10, 
            batch_size=128, 
            verbose=1, 
            validation_data=(x_val, y_val),
            callbacks=[tb_callback])
    
    #Plot Accuracy
    scores = model.evaluate(x=x_val, y=y_val)
    
    results["val_scores"].append(scores)
    results["params"].append(param)

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
best_run_idx = np.argmax(results["val_scores"])
results["best_score"] = results["val_scores"][best_run_idx]
results["best_params"] = results["params"][best_run_idx]
print(f"Best score: {results['best_score']} using params: {results['best_params']} \n")

scores = results["val_scores"]
params = results["params"]
for score, param in zip(scores, params):
    print(f"Score: {score} using params: {param}")


Download Model/Weights

In [ ]:
#weights
model.save_weights(filepath=MODEL_FILE_PATH)
model.save(filepath=FULL_MODEL_FILE_PATH)

Load Model/Weights

In [ ]:
model.load_weights(filepath=MODEL_FILE_PATH)
model = load_model(filepath=FULL_MODEL_FILE_PATH)